In [3]:
import sys
import os
from typing import Text
from absl import logging
from tfx.orchestration import metadata, pipeline
from tfx.orchestration.beam.beam_dag_runner import BeamDagRunner

PIPELINE_NAME = "acqmal-pipeline"

# pipeline inputs
DATA_ROOT = "/content/data"
UTILS_MODULE_FILE = "modules/utils_module.py"

# pipeline outputs
OUTPUT_BASE = "output"
serving_model_dir = os.path.join(OUTPUT_BASE, "serving_model")
pipeline_root = os.path.join(OUTPUT_BASE, PIPELINE_NAME)
metadata_path = os.path.join(pipeline_root, "metadata.sqlite")

def init_local_pipeline(components, pipeline_root: Text) -> pipeline.Pipeline:
    logging.info(f"Pipeline root set to: {pipeline_root}")
    beam_args = [
        "--direct_running_mode=multi_processing"
        "----direct_num_workers=0"
    ]

    return pipeline.Pipeline(
        pipeline_name=PIPELINE_NAME,
        pipeline_root=pipeline_root,
        components=components,
        enable_cache=True,
        metadata_connection_config=metadata.sqlite_metadata_connection_config(
            metadata_path
        ),
        # eam_pipeline_args=beam_args
    )

if __name__ == "__main__":
    logging.set_verbosity(logging.INFO)

    from modules.components import init_components

    components = init_components(
        DATA_ROOT,
        utils_module=UTILS_MODULE_FILE,
        training_steps=1000,
        eval_steps=1000,
        serving_model_dir=serving_model_dir
    )

    pipeline = init_local_pipeline(components, pipeline_root)
    BeamDagRunner().run(pipeline=pipeline)




INFO:absl:Finished tuning... Tuner ID: tuner0
INFO:absl:Best HyperParameters: {'space': [{'class_name': 'Int', 'config': {'name': 'embedding', 'default': None, 'conditions': [], 'min_value': 16, 'max_value': 128, 'step': 16, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'lstm_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Int', 'config': {'name': 'dense_units', 'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}}, {'class_name': 'Choice', 'config': {'name': 'learning_rate', 'default': 0.0001, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}}], 'values': {'embedding': 32, 'lstm_units': 128, 'dense_units': 384, 'learning_rate': 0.01}}
INFO:absl:Best Hyperparameters are written to output/acqmal-pipeline/Tuner/best_hyperparameters/31/best_hyperparameters.txt.
INFO:absl:Tuner results are written to output/acqmal-pipeline

Trial 50 Complete [00h 00m 16s]
accuracy: 0.30520832538604736

Best accuracy So Far: 0.375
Total elapsed time: 00h 42m 51s
Results summary
Results in output/acqmal-pipeline/Tuner/.system/executor_execution/31/.temp/31/emotion_kt
Showing 10 best trials
Objective(name="accuracy", direction="max")

Trial 31 summary
Hyperparameters:
embedding: 32
lstm_units: 128
dense_units: 384
learning_rate: 0.01
Score: 0.375

Trial 03 summary
Hyperparameters:
embedding: 128
lstm_units: 64
dense_units: 512
learning_rate: 0.01
Score: 0.37291666865348816

Trial 47 summary
Hyperparameters:
embedding: 96
lstm_units: 448
dense_units: 160
learning_rate: 0.0001
Score: 0.3453125059604645

Trial 34 summary
Hyperparameters:
embedding: 128
lstm_units: 480
dense_units: 96
learning_rate: 0.001
Score: 0.34166666865348816

Trial 00 summary
Hyperparameters:
embedding: 96
lstm_units: 32
dense_units: 224
learning_rate: 0.01
Score: 0.33906251192092896

Trial 05 summary
Hyperparameters:
embedding: 128
lstm_units: 256
dense_

INFO:absl:Execution 31 succeeded.
INFO:absl:Cleaning up stateful execution info.
INFO:absl:Deleted stateful_working_dir output/acqmal-pipeline/Tuner/.system/stateful_working_dir/ad3e4785-be01-4494-9604-9004057e0c8a
INFO:absl:Publishing output artifacts defaultdict(<class 'list'>, {'best_hyperparameters': [Artifact(artifact: uri: "output/acqmal-pipeline/Tuner/best_hyperparameters/31"
, artifact_type: name: "HyperParameters"
)], 'tuner_results': [Artifact(artifact: uri: "output/acqmal-pipeline/Tuner/tuner_results/31"
, artifact_type: name: "TunerResults"
)]}) for execution 31
INFO:absl:MetadataStore with DB connection initialized
INFO:absl:node Tuner is finished.
INFO:absl:node Trainer is running.
INFO:absl:Running launcher for node_info {
  type {
    name: "tfx.components.trainer.component.Trainer"
    base_type: TRAIN
  }
  id: "Trainer"
}
contexts {
  contexts {
    type {
      name: "pipeline"
    }
    name {
      field_value {
        string_value: "acqmal-pipeline"
      }
    

Model: "outputs"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 text_xf (InputLayer)        [(None, 1)]               0         
                                                                 
 text_vectorization (TextVe  (None, 100)               0         
 ctorization)                                                    
                                                                 
 embedding (Embedding)       (None, 100, 32)           32000     
                                                                 
 bidirectional_1 (Bidirecti  (None, 256)               164864    
 onal)                                                           
                                                                 
 dense_2 (Dense)             (None, 384)               98688     
                                                                 
 dense_3 (Dense)             (None, 6)                 2310

INFO:absl:Feature label has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Feature text has a shape dim {
  size: 1
}
. Setting to DenseTensor.
INFO:absl:Function `serve_tf_examples_fn` contains input name(s) table_handle, 1063976, resource with unsupported characters which will be renamed to outputs_text_vectorization_string_lookup_none_lookup_lookuptablefindv2_table_handle, outputs_embedding_embedding_lookup_1063976, outputs_dense_3_biasadd_readvariableop_resource in the SavedModel.
INFO:absl:Found untraced functions such as _update_step_xla, lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses, lstm_cell_layer_call_fn, lstm_cell_layer_call_and_return_conditional_losses while saving (showing 5 of 5). These functions will not be directly callable after loading.
INFO:absl:Writing fingerprint to output/acqmal-pipeline/Trainer/model/32/Format-Serving/fingerprint.pb
INFO:absl:Training complete. Model written to output/acqmal-pipeline/Trainer/model/32/

In [6]:
!pip install pylint

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.9/521.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 274.6/274.6 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.3/92.3 kB 7.6 MB/s eta 0:00:00


In [8]:
!pylint modules/components.py

************* Module components
modules/components.py:53:0: C0303: Trailing whitespace (trailing-whitespace)
modules/components.py:112:0: C0303: Trailing whitespace (trailing-whitespace)
modules/components.py:126:0: C0303: Trailing whitespace (trailing-whitespace)
modules/components.py:141:0: C0303: Trailing whitespace (trailing-whitespace)
modules/components.py:142:0: C0305: Trailing newlines (trailing-newlines)
modules/components.py:22:0: R0914: Too many local variables (18/15) (too-many-locals)
modules/components.py:43:13: E1101: Module 'tfx.proto.example_gen_pb2' has no 'Output' member (no-member)
modules/components.py:44:19: E1101: Module 'tfx.proto.example_gen_pb2' has no 'SplitConfig' member (no-member)
modules/components.py:45:8: E1101: Module 'tfx.proto.example_gen_pb2' has no 'SplitConfig' member (no-member)
modules/components.py:46:8: E1101: Module 'tfx.proto.example_gen_pb2' has no 'SplitConfig' member (no-member)
modules/components.py:71:15: E1101: Module 'tfx.proto.traine

In [5]:
!zip output.zip -r output

  adding: output/ (stored 0%)
  adding: output/acqmal-pipeline/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/assets/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/variables/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/variables/variables.index (deflated 62%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/variables/variables.data-00000-of-00001 (deflated 6%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/fingerprint.pb (stored 0%)
  adding: output/acqmal-pipeline/Pusher/pushed_model/34/saved_model.pb (deflated 89%)
  adding: output/acqmal-pipeline/Pusher/.system/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/.system/executor_execution/ (stored 0%)
  adding: output/acqmal-pipeline/Pusher/.system/executor_executi